### Build simple LLM using Groq

In [25]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv() ## loading all the environment variables from the .env file

True

In [15]:
model = ChatGroq(model="llama-3.1-8b-instant")
model.invoke("what is the capital of india?")

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 42, 'total_tokens': 51, 'completion_time': 0.006906185, 'completion_tokens_details': None, 'prompt_time': 0.002000321, 'prompt_tokens_details': None, 'queue_time': 0.00547224, 'total_time': 0.008906506}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--ff476af9-759b-4419-8a4c-b9d1ae569727-0', usage_metadata={'input_tokens': 42, 'output_tokens': 9, 'total_tokens': 51})

Using messages we can interact with LLMs

In [16]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="Hello, how are you?")
]

In [19]:
response = model.invoke(messages)


In [21]:
# with string output parser
parser = StrOutputParser()
parser.invoke(response)

"Bonjour, je vais bien, merci. Comment puis-je vous aider aujourd'hui ?"

In [22]:
# using langchain expression language

chain = model | parser
chain.invoke(messages)

"Bonjour, comment allez-vous ? \n\n(I'm doing well, thank you for asking. How about you?)"

In [31]:
# prompt template

generic_template = "Translate following into {language}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("human", "{text}"),
    ]
)

In [33]:
result = prompt.invoke({"language": "French", "text": "Hello, how are you?"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [34]:
result.to_messages()

[SystemMessage(content='Translate following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

If you see the messages above is equivalent to the output of this above output, this is the power of chat prompt template

In [35]:
# new let us create a full chain

final_chain = prompt | model | parser
final_chain.invoke({"language": "French", "text": "Hello, how are you?"})

'Bonjour, comment ça va ?'

Here let me explain step by step:
- Prompt is being created using ChatPromptTemplate 
- Model we had created above using Groq's llama3.1
- We have StrOutputParser

We individually invoked them, we were able to get the output.

Here the final_chain we created will work like this:
- Take the input we have passed while calling .invoke({...})
- First parser - will internally call parser.invoke({...}) - gives messages as output
- These messages are passed as input to model, model.invoke(messages) internally gets called
    - output from this step => AIMessage(content="....")
- This above output is passed to parser, parser internally calls parser.invoke(AIMessage(content="...."))
    - output is "...."